# performance evaluation purpose

# Library import, etc

In [ ]:
# created based on cellular-05-new-weekend-modified-r6-ISD.ipynb

%reset -f 

In [ ]:
import datetime
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

import random
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn import ensemble
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import sklearn.model_selection as ms

from sklearn.model_selection import GridSearchCV
from sklearn import neighbors

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler 

from sklearn import svm
from sklearn.svm import SVC, SVR

import statsmodels.api as sm 

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

import geopy.distance

from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot

from statsmodels.graphics.tsaplots import plot_acf

# taken from https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/ 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM 
    # note: LSTM, if no activation is passed, then default seems to be sigmoid. https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
    # lsw) this doesn't seem to be tf.keras. Rather, this just seems to be keras. 
    
from keras import optimizers
from keras import callbacks    
from itertools import cycle 
import re
import pickle

In [ ]:
random_state_ = 108
seed_ = 9
np.random.seed(seed_)  # this is numpy
random.seed(seed_) # this is python module 

print( np.random.get_state()[1][0] ) # lsw) At this moment, seed_ will be printed. But then, seed will keep changing 

drop_col_th = 0.2 # drop columns by this ratio of NA or null 

In [ ]:
now = datetime.datetime.now()
fname_part = now.strftime("%Y%m%d_%H%M%S") 

# functions

# series_to_supervised

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, mode_=1):
    # n_in: Number of lag observations as input (X). Values may be between [1..len(data)] Optional. Defaults to 1.
    # n_out: Number of observations as output (y). Values may be between [0..len(data)-1]. Optional. Defaults to 1.    
    # mode_ = 1 => var, var2,..., else: 'user', 'bytes'....
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data) # convert to a dataframe 
    cols, names = list(), list() # empty list 
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        if mode_ == 1: 
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        else: 
            names += [('%s(t-%d)' % (c_name, i)) for c_name in list(data.columns) ]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if mode_ == 1: 
            if i == 0:
                names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
            else:
                names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
        else: 
            if i == 0:
                names += [('%s(t)' % (c_name)) for c_name in list(data.columns)]
            else:
                names += [('%s(t+%d)' % (c_name, i)) for c_name in list(data.columns)]
            
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def dto_str_wo_yr_sec(dto_in):
    # Given a datetime object as input, output a string w/ year, sec removed
    # input: datetime object
    # output: string
    
    return dto_in.strftime( "%m-%d %H:%M" )

In [ ]:
def Filter(string, substr): 
    import re
    
    return [str for str in string if
             any(sub in str for sub in substr)] 

In [ ]:
def list_rem_red(list_):
    # remove redundancy from a list, while keeping order
    u_list = [] # unique list
    [u_list.append(item) for item in list_ if item not in u_list]
    
    return u_list

In [ ]:
def list_diff(list1, list2):
    out = [item for item in list1 if not item in list2]
    return out

In [ ]:
# https://stackoverflow.com/questions/14720331/how-to-generate-random-colors-in-matplotlib

def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [ ]:
thre = float(input('Enter threshold for num_signchange(0-1):  '))

def num_signchange(x):
    # x is a series 
    
    if x.empty:
        return None
    else: 
        # https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python/21171725
        y = np.array(x)
        y_ = np.array(y[1:]) - np.array(y[0:-1])
        pos1 = np.where(np.diff(np.signbit( y_ )))[0] # find where sign changes occur, not the latter posion but the front position 
        pos2 = np.nonzero( np.abs(y_)> thre )[0]  # assuming minmax scaled, I assumed max 1 x 0.2 as the slope criterion

        return len( np.intersect1d(pos1, pos2) )

# Enter conditions

In [ ]:
###################################################################
# Used for df_traffic_set (subset around t_bs_i), EDA, performance estimation, plot df_traffic_stat_geo
###################################################################
    
adj_bs_choice = 3 # int(input('Enter adj bs choice for bs_set/df_traffic_set and then LSTM, 1(no adj bs), 2(with max_n_adj_bs bs), 3(input BS index manually): '))

max_n_adj_bs = 7 # int(input('Enter max_n_adj_bs(default 7) for reading or calculation purpose(incl center cell):  '))
max_n_adj_bs_rest = -1 # int(input('(for df_traffic_set) To adopt # of adj bs is smaller than max_n_adj_bs, enter the number, else enter -1(incl center cell):  '))
    
input_choice = 1 # int( input('Input(used for df_traffic_set generation), 1(users), 2(pkt), 3(bytes), 4(users, pkt), 5(users, bytes), 6(pkt, bytes), 7(all):  ') )

wknd_in_input = 2 # int(input('Include weekend info in the input_col? Enter 1 to do, 2 not to:  '))

pred_target_choice = 1 # int( input('Predict target(sh be subset of input_col), 1(users), 2(pkt), 3(bytes), 4(users, pkt), 5(users, bytes), 6(pkt, bytes), 7(all):  ') )

read_df_dist_m = 1 # int(input('Enter 1 to read df_dist_m and df_adj_bs, 2 to calc and save:  '))

DoKmeansCluster = 2 # int(input('Enter 1 to do Kmeans clustering, 2 to load previous results, 3 to load tdomain clustering: '))  
    
Do_perf_est = 1 # int(input('Enter 1 to do LSTM evaluation, 2 not to:  '))    

tr_ratio = 0.7 # float(input('Enter training ratio(def:0.7):  '))

# parameter input for reframing 
n_in_time = int(input('Enter n_in_time(>1 for multiple lag input):  ')) # 1 means, (t-1) only, 2 means (t-2) and (t-1)
n_out_time = int(input('Enter n_out_time:  ')) # 1 means t only, 2 means t and t+1

use_early_stop = 2 # int(input('Enter 1 to use early stopping:  '))

cl_i = int(input('Enter cluster index for analysis: '))

bs_list_cl_sset_sz = int(input('Enter bs_list_cl_sset_sz:  '))

n_iter = int(input('Enter n_iter for LSTM iteration(default is 5): '))
learn_rate_ = 0.001 # float(input('Enter learning rate(def is 0.001:  '))   

In [ ]:
input_ch_lst = [ ['NULL'], ['users'], ['packets'], ['bytes'], ['users', 'packets'], ['users', 'bytes'], ['packets', 'bytes'], ['users', 'packets', 'bytes'] ]
pred_tg_ch_lst = input_ch_lst.copy()

input_col = input_ch_lst[input_choice].copy()

if wknd_in_input==1:
    input_col +=['weekend']

pred_target_col = pred_tg_ch_lst[pred_target_choice]

shorten_bs = 1 # int(input('Enter 1 to restrict bs list by longitude/latitude range:  '))

# Kmeans clustering #1 (simple), based on df_traffic_FE

In [ ]:
if DoKmeansCluster == 2:
    pkl_name = 'kmeans__hvch_2_clsz_9_stch_9_20210315_003517.pkl' # input('Enter pkl_name with data path: ')
    df_traffic_FE_cl_name = 'df_traffic_FE_hvch_2_clsz_9_stch_9_20210315_003517.csv' # input('Enter df_traffic_FE_cl_name with path:  ')
        
    # https://stackoverflow.com/questions/54879434/how-to-use-the-pickle-to-save-sklearn-model
    kmeans = pickle.load(open(pkl_name, "rb"))
    df_traffic_FE_cluster = pd.read_csv(df_traffic_FE_cl_name, header=0)
    
    print('\n ====== kmeans.cluster_centers_ ======')
    print(kmeans.cluster_centers_[:, 0:10])
    print(kmeans.cluster_centers_.shape)

    print('\n kmeans.labels_[0:10]')
    print(kmeans.labels_[0:15])
    print( pd.Series( kmeans.labels_).value_counts() )
    
elif DoKmeansCluster == 3: # load tdomain clustering 
    # df_traffic_FE_cluster = pd.read_csv("AlexM_KMClust_7_dftrafficsc_users_r1.csv", header=0)
    df_traffic_FE_cluster_fname = input('Enter df_traffic_FE_cluster_fname including path:  ')
    df_traffic_FE_cluster = pd.read_csv(df_traffic_FE_cluster_fname, header=0)


In [ ]:
bs_list_cl = list( df_traffic_FE_cluster[df_traffic_FE_cluster.cluster==cl_i].bs )

np.random.seed(seed_)  # this is numpy # do it again, so that .choice() is reproducible

if cl_i == 0:
    bs_list_cl_sset = [6684, 5713, 3255, 4008]
elif cl_i == 1:     
    # bs_list_cl_sset = [4180, 8827, 3789, 8512] 
    bs_list_cl_sset = [4180, 3789, 8512, 3192] 
elif cl_i == 2:
    bs_list_cl_sset = [7198, 7749, 7778, 6277]
elif cl_i == 3:
    bs_list_cl_sset = [     ]   
elif cl_i == 4:
    bs_list_cl_sset = [5317, 5318, 5880, 6042]
elif cl_i == 5:
    bs_list_cl_sset = [7016, 6032, 6573, 7664]        
elif cl_i == 6:        
    bs_list_cl_sset = [3098, 1059, 11382, 4515]
elif cl_i == 7: 
    bs_list_cl_sset = [      ]      
elif cl_i == 8:        
    bs_list_cl_sset = [3110, 8349, 8482, 13110]

# df_traffic and df_toplogy are read

In [ ]:
# datetime.datetime.fromtimestamp( int(x) ) => returns ex) datetime.datetime(2012, 8, 19, 1, 0), a datetime object 
# x: numpy.int64 object 
    
def parse03(x):
    return datetime.datetime.fromtimestamp( int(x) )
        # returned example: 2012-08-19 01:00:00
        # Otherwise, directly from the csv file, time_hour column would be like: ex) 1345305600
        # Looks like, int(x) works same as just x 
        # datetime.fromtimestamp: Return the local date and time corresponding to the POSIX timestamp, 
        # such as is returned by...
        # csv file's Time_hour definition: hourly timestamp in UNIX epoch time (time zone GMT+8).

df_traffic = pd.read_csv("cellular_traffic_full.csv", \
                     parse_dates = [1],date_parser = parse03, header=0)

df_topology = pd.read_csv("topology.csv", header = 0, index_col=0)
    # note) longitude, latitude: usually units are in degrees # bs_i works as index    
    
print( df_traffic.describe() )
print('\n')
print( df_topology.describe() )

# df_dist_m, df_adj_bs generated, using df_topology
## of bs x max_n_adj_bs

In [ ]:
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
# The answers above are based on the Haversine formula, which assumes the earth is a sphere, 
# which results in errors of up to about 0.5% (according to help(geopy.distance)). 
# Vincenty distance uses more accurate ellipsoidal models such as WGS-84, and is implemented in geopy. 

# Update: 04/2018: Note that Vincenty distance is deprecated since GeoPy version 1.13 - you should use geopy.distance.distance() instea

def my_geopy_dist_m(lat1, long1, lat2, long2):
    # return geopy.distance.vincenty([lat1,long1], [lat2, long2]).m
    return geopy.distance.distance([lat1,long1], [lat2, long2]).m

# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula
def my_geopy_dist_list2_m(lat1, long1, lat2, long2):
    # lat1, long1, lat2, long2: should be in radians 
    R_km = 6371
    diff_long = np.radians(long2 - long1)
    diff_lat = np.radians(lat2 - lat1)
    a = math.sin(diff_lat/2)**2 + math.cos(np.radians(lat1)) * math.cos(np.radians(lat2)) * math.sin(diff_long/2)**2
    b = 2 * math.asin( np.minimum(1, np.sqrt(a))) 
    d = R_km * b
    return d

In [ ]:
# whether adj_bs_choice == 1 or 2, we setup assuming  = 7 here 

header_ = []
for ii in range(0,max_n_adj_bs-1): # -1, since max_n_adj_bs includes center cell
    header_ = header_ + ['adj_bs_i_' + str(ii)]
    
if read_df_dist_m==2: # then calculate     
    df_dist_m = pd.DataFrame(index=df_topology.index, columns=header_) # bs_i works as index
    df_adj_bs = pd.DataFrame(index=df_topology.index, columns=header_) # bs_i works as index   
elif read_df_dist_m==1: # then read
    if max_n_adj_bs ==7:
        df_dist_m = pd.read_csv( 'df_dist_m.csv', index_col=0, header = 0)
        df_adj_bs = pd.read_csv( 'df_adj_bs.csv', index_col=0, header = 0)
    else: 
        df_dist_m = pd.read_csv( 'df_dist_m_'+str(max_n_adj_bs)+'.csv' , index_col=0, header = 0)
        df_adj_bs = pd.read_csv( 'df_adj_bs_'+str(max_n_adj_bs)+'.csv' , index_col=0, header = 0)
        
    df_dist_m = df_dist_m.iloc[:, 0:max_n_adj_bs-1] # ex) if max_n_adj_bs=7, 0:6 ==> means neighboring 6 cells 

In [ ]:
if read_df_dist_m==2: # then calculate 
    start_time = datetime.datetime.now() 

    for bs_i in df_topology.index: 
        # for bs_i in range(1,2):
        lon_ = float( df_topology.loc[ df_topology.index==bs_i].lon )
        lat_ = float( df_topology.loc[ df_topology.index==bs_i].lat )
        lon_rep = np.repeat(lon_, df_topology.shape[0]-1)
        lat_rep = np.repeat(lat_, df_topology.shape[0]-1)
        lon_exc = list( df_topology.loc[ df_topology.index!=bs_i].lon )
        lat_exc = list( df_topology.loc[ df_topology.index!=bs_i].lat )

        dist_m_vec = list( map( my_geopy_dist_m, lat_rep, lon_rep, lat_exc, lon_exc) ) # => takes 2.8 sec 
            # or use: dist_m_vec = [ geopy.distance.distance([x,y], [z, w]).m for x,y,z,w in zip( lat_rep, lon_rep, lat_exc, lon_exc ) ]
        adj_bs_vec = list(  set(df_topology.index)-set([bs_i])  ) 
            # using set and not being able to preserve order is OK here. 
        
        idx = np.argsort( dist_m_vec ) # sort in ascending order => then find the idx
        idx = idx[0:max_n_adj_bs-1] # take only max_n_adj_bs elements # -1 since, max_n_adj_bs means including center cell 

        dist_m_vec = [dist_m_vec[i] for i in idx] # take only max_n_adj_bs -1 elements 
        adj_bs_vec = [adj_bs_vec[i] for i in idx] # take only max_n_adj_bs -1 elements 

        df_dist_m.loc[ df_dist_m.index==bs_i,:]=dist_m_vec
        df_adj_bs.loc[ df_adj_bs.index==bs_i,:]=adj_bs_vec

        if bs_i % 30 == 0: 
            time_elapsed = datetime.datetime.now() - start_time # start_time is fixed, was measured at the point of loop_i=0, time_elapsed keeps being updated
            print('Avg time consumed per iteration(hh:mm:ss.ms) is {}'.format(time_elapsed/(bs_i)) )    
            print('Time elapsed so far(hh:mm:ss.ms) is %s as of %s'%(format(time_elapsed),datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
            print('Remaiging time estimated(hh:mm:ss.ms) is %s as of %s' %(format(time_elapsed/(bs_i)*( max(df_topology.index)-bs_i )),datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) )   
            
    if max_n_adj_bs==7:
        df_dist_m.to_csv( 'city-cellular-traffic-map-master/traceset/df_dist_m.csv')
        df_adj_bs.to_csv( 'city-cellular-traffic-map-master/traceset/df_adj_bs.csv')
    else: 
        df_dist_m.to_csv( 'city-cellular-traffic-map-master/traceset/df_dist_m_' + str(max_n_adj_bs) +'.csv')
        df_adj_bs.to_csv( 'city-cellular-traffic-map-master/traceset/df_adj_bs_' + str(max_n_adj_bs) +'.csv') 

# df_traffic is further processed to incorporate weekend info or ISD info

In [ ]:
df_traffic['weekend']=list( (df_traffic.time_hour.dt.weekday==5) | (df_traffic.time_hour.dt.weekday==6) )

# df_info generation for df_traffic 

In [ ]:
n_unique =df_traffic.nunique() 
null_perc = df_traffic.isnull().sum()/df_traffic.shape[0]*100
NA_perc = df_traffic.isnull().sum()/df_traffic.shape[0]*100

df_info = pd.DataFrame({'n_unique': n_unique, 'null_perc': null_perc, 'NA_perc': NA_perc})
df_info["dtyp"]=""
df_info["iotyp"]=""

for r_i in range(0, df_info.shape[0] ):
    if df_info['n_unique'][r_i] >=10: # 10: just chosen 
        df_info['dtyp'][r_i] = 'numeric'
    elif df_info['n_unique'][r_i] ==2: # don't care, whether nominal or ordinal 
        df_info['dtyp'][r_i] = 'categorical'
    else:
        df_info['dtyp'][r_i] = 'nominal' 
        
    df_info["iotyp"][r_i]='inp'  
        
print(df_info)

# BS shortening
## df_topology, df_traffic are affected

In [ ]:
if shorten_bs == 1:
    lon_min = df_topology.describe().loc['mean','lon']-0.5*df_topology.describe().loc['std','lon']
    lon_max = df_topology.describe().loc['mean','lon']+0.5*df_topology.describe().loc['std','lon']
    lat_min = df_topology.describe().loc['mean','lat']-0.5*df_topology.describe().loc['std','lat']
    lat_max = df_topology.describe().loc['mean','lat']+0.5*df_topology.describe().loc['std','lat']
    
    print(df_topology.shape)
    
    df_topology = df_topology.loc[(df_topology['lon'] >= lon_min) & (df_topology['lon'] <= lon_max) & \
                    (df_topology['lat'] >= lat_min) & (df_topology['lat'] <= lat_max)]
    
    df_traffic = df_traffic.loc[ df_traffic.bs.isin(list(df_topology.index)) ]    
    
    print(df_topology.shape)
    print('\n')
    print( df_traffic.describe() )
    print('\n')
    print( df_topology.head(3) )
    print('\n')    
    print( df_topology.describe() )

In [ ]:
if shorten_bs == 1:
    print("================ df_traffic.head(3) ================")
    print(df_traffic.head(3))

    print("\n================ df_traffic.shape ================")
    print(df_traffic.shape)

    print("\n================ df_traffic.dtypes ================")
    print(df_traffic.dtypes)

    print("================ df_topology.head(3) ================")
    print(df_topology.head(3))

    print("\n================ df_topology.shape ================")
    print(df_topology.shape)

    print("\n================ df_topology.dtypes ================")
    print(df_topology.dtypes)

In [ ]:
#########################################################################################################
## df_traffic_set (subset around t_bs_i) generated using bs_set
## There is NA within df_traffic_set, since starting time differs for BSs ==> Need to handle this.
#########################################################################################################

missing_proc = 1 # int(input('How to process missing values? 1(dropna), 2(fillna -1), 3(masking)'))

temp1 = np.tile( ['time_'], (1, n_out_time*len(pred_target_col) )  )[0]
temp2 = np.tile( np.array(range(0, n_out_time)) , (1,len(pred_target_col)) )[0]
temp3 = np.tile( ['_target_'], (1, n_out_time*len(pred_target_col) )  )[0]
temp4 = np.tile( np.array( [list( range(0, len(pred_target_col)) )] ).transpose(), (1, n_out_time) ).reshape(-1, len(pred_target_col)*n_out_time )[0]
rmse_c = ['bs1', 'bs2']
for ii in range(0,len(temp1)):
    rmse_c.append( temp1[ii]+str(temp2[ii])+temp3[ii]+str(temp4[ii]) )
del temp1, temp2, temp3, temp4
df_rmse_arr_avg = pd.DataFrame(columns = rmse_c) 
df_rmse_arr_nsc_avg = pd.DataFrame(columns = rmse_c) 

start_time2 = datetime.datetime.now()

n_neuron = int(input('Enter # of neurons for LSTM(default 50):  '))
batch_size_ = int(input('Enter batch_size(def:288)'))
n_epochs = int(input('Enter n_epochs(default 100): '))

###################################
for bs_ii in range(0, len(bs_list_cl_sset) ):
###################################    
    for bs_jj in range(0, len(bs_list_cl_sset) ):
        t_bs_i = bs_list_cl_sset[bs_ii] # cell where performance needs to be evaluated
        if bs_ii == bs_jj: 
            bs_set = [ bs_list_cl_sset[bs_ii] ]
        else: 
            bs_set = [ bs_list_cl_sset[bs_ii] , bs_list_cl_sset[bs_jj] ]
        
        col_names = []
        idx_ = 0

        for bs_i in bs_set:
            if bs_i==bs_set[0]:
                df_traffic_set = df_traffic.loc[df_traffic.bs==bs_i, ['time_hour'] + input_col ]
                col_names = col_names + ['time_hour'] + list( map( lambda a,b: a+b, input_col , ['_' + str(bs_i)]*len(input_col) ) )
            else:
                idx_ = idx_ + 1        
                
                df_traffic_set = pd.merge(df_traffic_set, df_traffic.loc[df_traffic.bs==bs_i, ['time_hour']+input_col ] \
                                          , how='outer', on ='time_hour' )
                col_names = col_names + list( map( lambda a,b: a+b, input_col, ['_' + str(bs_i)]*len(input_col) ) )
            
        df_traffic_set = df_traffic_set.sort_values(by = 'time_hour', axis=0, ascending=True ) # otherwise, merge was unsorted sometimes     
        df_traffic_set.columns = col_names # use the completed col_names

        null_perc = df_traffic_set.isnull().sum()/df_traffic_set.shape[0]*100
        NA_perc = df_traffic_set.isnull().sum()/df_traffic_set.shape[0]*100

            
        null_perc_set = df_traffic_set.isnull().sum()/df_traffic_set.shape[0]*100
        print("================= before droping columns =============== ")
        print( null_perc_set )
        print( df_traffic_set.shape )

        df_traffic_set = df_traffic_set.iloc[:, list(null_perc_set < drop_col_th*100 ) ]

        null_perc_set = df_traffic_set.isnull().sum()/df_traffic_set.shape[0]*100
        print("================= after droping columns =============== ")    
        print( null_perc_set )
        print( df_traffic_set.shape )        
        
        df_traffic_set = df_traffic_set.set_index('time_hour') # W/o this command, interpolate() encounters errors 
        interp_number = 20 # int(input('Enter interp_number:  '))

        df_traffic_set = df_traffic_set.resample(str(interp_number)+'T').interpolate()
        # ref for resample: https://machinelearningmastery.com/resample-interpolate-time-series-data-python/ 
        # T works as one minte # '3T': 3 minutes# '30S': 30 seconds 

        if missing_proc == 1:
            df_traffic_set = df_traffic_set.dropna(axis=0, how='any') # row is remoed by any 
        elif missing_proc == 2:
            df_traffic_set.fillna(-1000, inplace=True)
        elif missing_proc == 3:    
            df_traffic_set.fillna(-1000, inplace=True)

        index_ = list(df_traffic_set.index)
        columns_ = list(df_traffic_set.columns)
        scaler = MinMaxScaler(feature_range=(0, 1))

        df_traffic_set = pd.DataFrame(data = scaler.fit_transform( df_traffic_set.values ), index=index_, columns=columns_) # bs_i works as index
        del index_
        del columns_
        
        df_traffic_set_ref = series_to_supervised( df_traffic_set, n_in_time, n_out_time, True, 2 ) # shape is (43799, 16) # a dataframe 
        
        var_list_in3 = [x for x in list(df_traffic_set_ref.columns) if 't-' in x]

        dd = [elem in pred_target_col for elem in input_col] # ex) [True, True, False]
        dd_idx = [i+1 for i in range(len(dd)) if dd[i] == True] # make it a list of index # +1 so that 1,2,3 instead of 0,1,2
        dd_idx_str = ["{}".format(element) for element in dd_idx] # ex) ['1', '2', '3']

        var_list_out3 = [x for x in list(df_traffic_set_ref.columns) if ('t-' not in x)]
        var_list_out3 = Filter(var_list_out3, pred_target_col) # out of var_list_out3, take only ones contraiing elements of pred_target_col
        var_list_out3 = Filter(var_list_out3, [str(t_bs_i)] )
        
        drop_pos = list( set(df_traffic_set_ref.columns) - set(var_list_in3) - set(var_list_out3) )

        df_traffic_set_ref.drop( drop_pos, axis=1, inplace=True ) 
        
        n_train_samples = round(df_traffic_set_ref.shape[0]*tr_ratio)

        train = df_traffic_set_ref.iloc[:n_train_samples, :] # both x and y 
        test = df_traffic_set_ref.iloc[n_train_samples:, :] # both x an y 
       
        train_X, train_y = train[var_list_in3], train[var_list_out3]
        test_X, test_y = test[var_list_in3], test[var_list_out3]     
        
        train_X = train_X.values.reshape(( train_X.shape[0], n_in_time, len(input_col)*len(bs_set) ))
        test_X = test_X.values.reshape((   test_X.shape[0],  n_in_time, len(input_col)*len(bs_set) ))        
        
        stacked_LSTM = 1 # int(input('Enter 1 for one additiional stacked LSTM layer, 2 for two additinal LSTM layers:  '))
        if stacked_LSTM == 1:
            n_neuron_stacked = 20 # int(input('n_neuron_stacked:  '))
        elif stacked_LSTM == 2:
            n_neuron_stacked = int(input('n_neuron_stacked:  '))
            n_neuron_stacked2 = int(input('n_neuron_stacked2:  '))

        acti_fn = 1# int(input('enter 1 for sigmoid, 2 for relu: '))
        if acti_fn == 1:
            acti_fn = 'sigmoid'
        elif acti_fn == 2:
            acti_fn = 'relu'
        
        rmse_arr = np.zeros(( n_iter, n_out_time, len(pred_target_col)) )
        rmse_arr_nsc  = np.zeros(( n_iter, n_out_time, len(pred_target_col)) )
        
        ###################################
        for iter_ii in range(0,n_iter):
        ###################################    
            # design network
            model = Sequential()
            if stacked_LSTM in [1]: # https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ => stacked LSTM
                model.add(LSTM(n_neuron, activation=acti_fn, return_sequences=True, \
                               input_shape=(train_X.shape[1], train_X.shape[2]))) 
                    # [ timesteps, features]. 3 50 neurons
                    # or n_in_time, len(input_col) 
                    # 50: a hidden layer with 4 LSTM blocks or neurons,    
                    # return_sequences=True : https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ => stacked LSTM
                model.add( LSTM(n_neuron_stacked, activation=acti_fn) )
                model.add(Dense(n_out_time))
            elif stacked_LSTM in [2]:   
                # https://machinelearningmastery.com/stacked-long-short-term-memory-networks/
                model.add(LSTM(n_neuron, activation=acti_fn, return_sequences=True, \
                               input_shape=(train_X.shape[1], train_X.shape[2]))) 
                model.add( LSTM(n_neuron_stacked, activation=acti_fn, return_sequences=True) )
                model.add( LSTM(n_neuron_stacked2, activation=acti_fn) )
                model.add(Dense(n_out_time))

            else: 
                model.add( LSTM(n_neuron, activation=acti_fn, input_shape=(train_X.shape[1], train_X.shape[2])) ) 
                    #  [ timesteps, features]. 3 50 neurons
                    # or n_in_time, len(input_col) 
                    # 50: a hidden layer with 4 LSTM blocks or neurons,
                model.add(Dense(n_out_time))

            # https://keras.io/api/optimizers/adam/
                # tf.keras.optimizers.Adam( learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name="Adam", **kwargs)
            # https://datascience.stackexchange.com/questions/48736/how-to-see-change-learning-rate-in-keras-lstm
                # sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
                # model.compile(loss='mean_squared_error', optimizer=sgd)
            adam_ = optimizers.Adam(lr = learn_rate_ )
            model.compile( loss='mae', optimizer=adam_ )

            # fit network
            es_callback = callbacks.EarlyStopping(monitor='val_loss', patience=3)
            if use_early_stop==1: 
                # https://towardsdatascience.com/dont-overfit-how-to-prevent-overfitting-in-your-deep-learning-models-63274e552323
                history = model.fit(train_X, train_y, epochs=n_epochs, batch_size=batch_size_, \
                                validation_data=(test_X, test_y), verbose=1, shuffle=False, callbacks=[es_callback] )   
            else: 
                history = model.fit(train_X, train_y, epochs=n_epochs, batch_size=batch_size_, \
                                validation_data=(test_X, test_y), verbose=1, shuffle=False )

            # plot history # https://keras.io/ko/visualization/ or 
            pyplot.plot(history.history['loss'], label='train')
            pyplot.plot(history.history['val_loss'], label='test')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            pyplot.legend()
            pyplot.show()

            test_y_hat = model.predict(test_X) # 2D ndarray is returned

            test_X_resh = test_X.reshape((test_X.shape[0], n_in_time*len(input_col)*len(bs_set) )) # 3D => 2D 
            # back to 2D dataframe 
            test_X_resh = pd.DataFrame(data = test_X_resh, columns=var_list_in3 ) # bs_i works as index
            test_y_hat  = pd.DataFrame(data=test_y_hat, columns = var_list_out3 )


            for t_i in range(0,n_out_time): # time index
                for v_i in range(0, len(pred_target_col)): # variable index
                    input_1   = test_X_resh.iloc[ :, 0:len(input_col)*len(bs_set) ] 
                    input_1_1 = test_X_resh.iloc[ :, 0:len(input_col)*len(bs_set) ] 

                    # from test_y_hat => make a column 
                    if t_i == 0: 
                        input_2   = test_y_hat[  [pred_target_col[v_i]+'_'+str(t_bs_i)+'(t)'] ]
                        input_2_1 = list( test_y[ pred_target_col[v_i]+'_'+str(t_bs_i)+'(t)' ] )
                            # Why list()? Othewise, input_2_1 would be a dataframe with time as index, unlike input_2
                            # Then, plugging into input_1_1 will cause problem 
                    else: 
                        input_2   = test_y_hat[  [pred_target_col[v_i]+'_'+str(t_bs_i)+'(t+' + str(t_i) + ')'] ]
                        input_2_1 = list( test_y[ pred_target_col[v_i]+'_'+str(t_bs_i)+'(t+' + str(t_i) + ')' ] )
                            # othewise, input_2_1 would be a dataframe with time as index, unlike input_2
                            # Then, plugging into input_1_1 will cause problem 

                    input_1.iloc[:, dd_idx[v_i]-1]   = input_2   # -1 since, dd_idx was generated with +1 
                    input_1_1.iloc[:, dd_idx[v_i]-1] = input_2_1 # -1 since, dd_idx was generated with +1 

                    inv_test_y_hat = scaler.inverse_transform(input_1) # in/out would be same size as df_traffic_set
                    inv_test_y_hat = inv_test_y_hat[:, dd_idx[v_i]-1 ] # slice out only this position

                    inv_test_y     = scaler.inverse_transform(input_1_1) # in/out would be same size as df_traffic_set
                    inv_test_y     = inv_test_y[:, dd_idx[v_i]-1 ] # slice out only this position

                    rmse_arr[iter_ii, t_i, v_i] = np.sqrt(sklearn.metrics.mean_squared_error(inv_test_y, inv_test_y_hat))
                    rmse_arr_nsc[iter_ii, t_i, v_i] = np.sqrt(sklearn.metrics.mean_squared_error(input_2, input_2_1))                    
                    print('Test RMSE: %.3f' % rmse_arr[iter_ii, t_i, v_i] ) 

                    if iter_ii==0 and t_i == n_out_time-1: ## plot only once
                        t_axis =  df_traffic_set_ref.index[n_train_samples:]
                        plt.plot(t_axis, inv_test_y, 'r', label='inv_test_y') # plotting t, a separately 
                        plt.plot(t_axis, inv_test_y_hat, 'b', label='inv_test_y_hat') # plotting t, b separately 

                        title_ =  'var_' + pred_target_col[v_i] + '(t+' + str(t_i) + ')'

                        plt.title(title_)
                        plt.legend()
                        plt.show()       

                    if iter_ii==0 and t_i==n_out_time-1: ## plot only once
                        t_axis =  df_traffic_set_ref.index[n_train_samples:]

                        plt.plot(t_axis, test_y.iloc[:, [t_i*len(pred_target_col)+v_i]], 'r', label='test_y(t)') # plotting t, a separately 
                        plt.plot(t_axis, test_y_hat.iloc[:, [t_i*len(pred_target_col)+v_i]], 'b', label='test_y_hat(t)') # plotting t, b separately 
     
                        temp1 = test_y_hat.iloc[ test_y_hat.shape[1]-1:, [0] ] # (t) with delay
                        temp2 = np.zeros( [test_y_hat.shape[1]-1,1] ) 

                        plt.plot(t_axis, np.concatenate( (temp1, temp2), axis= 0 )    , 'k', label='test_y_hat(t+delay)') # plotting t, b separately 

                        title_ =  'var_' + pred_target_col[v_i] + '_shifted'
                        plt.title(title_)
                        plt.legend()
                        plt.show()                  

            # calculated and printed per iter_ii
            time_elapsed2 = datetime.datetime.now() - start_time2 # start_time is fixed, was measured at the point of loop_i=0, time_elapsed keeps being updated
            if bs_jj - (bs_ii-1) != 0:
                avg_time = time_elapsed2/(iter_ii+1)/(bs_ii+1)/(bs_jj - (bs_ii-1))
                rem_time = avg_time * ( len(bs_list_cl_sset)-1 - (bs_ii-1) ) * ( len(bs_list_cl_sset)-1 - (bs_jj-1) )*( n_iter-1 - (iter_ii-1) )
                print('Avg time consumed per iteration(hh:mm:ss.ms) is {}'.format( avg_time ) )    
                print('Time elapsed so far(hh:mm:ss.ms) is %s as of %s'%(format(time_elapsed2),datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
                print('Remaiging time estimated(hh:mm:ss.ms) is %s as of %s' %(format( rem_time ),datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) )              
  
        # end of for iter_ii in range(0,n_iter):    
        
        rmse_arr_avg = np.average( rmse_arr, axis=0)  
        rmse_arr_nsc_avg = np.average( rmse_arr_nsc, axis=0)        
        
        if( len(bs_set)==1 ):
            df_rmse_arr_avg.loc[len(df_rmse_arr_avg)]         = bs_set + [-1] + list( rmse_arr_avg.flatten() ) 
            df_rmse_arr_nsc_avg.loc[len(df_rmse_arr_nsc_avg)] = bs_set + [-1] + list( rmse_arr_nsc_avg.flatten() ) 
        else: 
            df_rmse_arr_avg.loc[len(df_rmse_arr_avg)] = bs_set + list( rmse_arr_avg.flatten() ) 
            df_rmse_arr_nsc_avg.loc[len(df_rmse_arr_nsc_avg)] = bs_set + list( rmse_arr_nsc_avg.flatten() ) 
        
        print(rmse_arr_avg)

    # end of for bs_jj in range( bs_ii+1, len(bs_list_cl_sset) ):
# end of for bs_ii in range(0, len(bs_list_cl_sset) ):    



# summary of test results

In [ ]:
df_rmse_arr_avg

In [ ]:
fname = 'df_rmse_arr_avg_' + fname_part + '_cl_' + str(cl_i) + '_setsz_' + str(bs_list_cl_sset_sz) + '_iter_' + str(n_iter)+'.csv'

df_rmse_arr_avg.to_csv( fname )

fname2 = 'df_rmse_arr_nsc_avg_' + fname_part + '_cl_' + str(cl_i) + '_setsz_' + str(bs_list_cl_sset_sz) + '_iter_' + str(n_iter)+'.csv'

df_rmse_arr_nsc_avg.to_csv( fname2 )